# Compute (DLC) Pose Estimation accuracy
First, we'll import the hand labeled ground truth files.

This notebook is designed to work with the standard folder structure found in [DeepLabCut](https://github.com/DeepLabCut/DeepLabCut) projects. We are going to be using our own custom functions for evaluation here, so we're going to use DLC-Live so we can point to individual images rather than entire datasets with a subsection of labels.

In [1]:
from dlclive import DLCLive, Processor
import cv2

dlc_proc = Processor()
dlc_trained_model_path = "I:/FARTS/DeepLabCut_Multi_Sticks_V1/multi_stick_v1_FARTS-2021-08-04/exported-models/DLC_multi_stick_v1_resnet_50_iteration-0_shuffle-1"
dlc_live = DLCLive(dlc_trained_model_path, processor=dlc_proc)

now let's load all the data we want to evaluate!

This script needs to point at the location of the "labeled-data" folder of your DLC project, expecting the hierachy:

***labeled-data/VIDEOS/CollectedData_USER.h5***

In [2]:
import os
from pathlib import Path
import pandas as pd
labeled_data_path = Path("I:/BENCHMARK/Sungaya_hand_labeled")

# grab all h5 files
first_h5_found = False
for subdir, dirs, files in os.walk(labeled_data_path):
    for file in files:
        if file[-1] == "5":
            if not first_h5_found:
                df_labeled = pd.read_hdf(os.path.join(subdir, file))
                first_h5_found = True
            else:
                more_labeled = pd.read_hdf(os.path.join(subdir, file))
                df_labeled = df_labeled.append(more_labeled)

df_labeled

scorer                                     Hendrik                           \
bodyparts                                l_antenna                r_antenna   
coords                                           x           y            x   
labeled-data/spec2_cam0_d/img0726.png   140.369533  739.648848   103.072561   
labeled-data/spec2_cam0_d/img0749.png   357.464128  717.504572   347.479778   
labeled-data/spec2_cam0_d/img0750.png   367.380356  711.422910   357.834862   
labeled-data/spec2_cam0_d/img0780.png   805.068688  459.579159   879.173121   
labeled-data/spec2_cam0_d/img0782.png   852.903569  457.464509   922.602618   
...                                            ...         ...          ...   
labeled-data/spec7_cam4_i/img476.png   1737.339885  203.596400  1824.966020   
labeled-data/spec7_cam4_i/img490.png   1873.180917  363.783456  1935.135220   
labeled-data/spec7_cam4_i/img525.png           NaN         NaN          NaN   
labeled-data/spec7_cam4_i/img559.png           NaN         NaN          NaN   
labeled-data/spec7_cam4_i/img647.png           NaN         NaN          NaN   

scorer                                                                      \
bodyparts                                                 head               
coords                                          y            x           y   
labeled-data/spec2_cam0_d/img0726.png  959.479519          NaN         NaN   
labeled-data/spec2_cam0_d/img0749.png  947.031210   154.820928  822.031798   
labeled-data/spec2_cam0_d/img0750.png  936.524933   164.366369  813.146659   
labeled-data/spec2_cam0_d/img0780.png  744.778930   681.627438  649.355238   
labeled-data/spec2_cam0_d/img0782.png  726.569617   711.426452  637.981903   
...                                           ...          ...         ...   
labeled-data/spec7_cam4_i/img476.png   263.015207  1665.850587  343.625668   
labeled-data/spec7_cam4_i/img490.png   455.638521  1771.812008  353.201384   
labeled-data/spec7_cam4_i/img525.png          NaN  2035.843680  349.747242   
labeled-data/spec7_cam4_i/img559.png          NaN          NaN         NaN   
labeled-data/spec7_cam4_i/img647.png          NaN          NaN         NaN   

scorer                                                                       \
bodyparts                                       t1                       t2   
coords                                           x           y            x   
labeled-data/spec2_cam0_d/img0726.png          NaN         NaN          NaN   
labeled-data/spec2_cam0_d/img0749.png   111.885920  820.521995     3.085781   
labeled-data/spec2_cam0_d/img0750.png   119.152956  811.755478    12.379743   
labeled-data/spec2_cam0_d/img0780.png   636.772446  660.319792   537.343880   
labeled-data/spec2_cam0_d/img0782.png   670.367328  647.700039   564.196694   
...                                            ...         ...          ...   
labeled-data/spec7_cam4_i/img476.png   1623.356454  354.184816  1542.746371   
labeled-data/spec7_cam4_i/img490.png   1726.738306  358.002133  1654.193648   
labeled-data/spec7_cam4_i/img525.png   1988.705659  352.857873  1912.854122   
labeled-data/spec7_cam4_i/img559.png           NaN         NaN          NaN   
labeled-data/spec7_cam4_i/img647.png           NaN         NaN          NaN   

scorer                                             ...                 \
bodyparts                                          ... h_r_trochanter   
coords                                          y  ...              x   
labeled-data/spec2_cam0_d/img0726.png         NaN  ...            NaN   
labeled-data/spec2_cam0_d/img0749.png  802.121278  ...            NaN   
labeled-data/spec2_cam0_d/img0750.png  790.887749  ...            NaN   
labeled-data/spec2_cam0_d/img0780.png  668.543207  ...     454.503508   
labeled-data/spec2_cam0_d/img0782.png  655.474548  ...     489.545284   
...                                           ...  ...            ...   
labeled-data/spec7_cam4_i/img476.png

In [19]:
# let's check the labels of the imported annotated data
import numpy as np
point_size = 5

for item in df_labeled.iterrows():
    print(item[0], item[1][0])
    img_name = item[0]
    img_path = os.path.join(labeled_data_path,img_name)
    img = cv2.imread(str(img_path))

    for coord in range(int((len(item[1])) / 2)):
        if not np.isnan(item[1][coord*2]) and not np.isnan(item[1][coord*2 + 1]):
            img = cv2.circle(img, (int(item[1][coord*2]), int(item[1][coord*2 + 1])),
                                       point_size,
                                       (int(255 * coord / 49), int(255 - 255 * coord / 49), 200), -1)
                   
    cv2.imshow("DLC Labeled IMG",img)
    cv2.waitKey(1)
    
cv2.destroyAllWindows()

labeled-data/spec2_cam0_d/img0726.png 140.3695330583816
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam0_d/img0726.png
annotated limbs 46
labeled-data/spec2_cam0_d/img0749.png 357.46412826024755
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam0_d/img0749.png
annotated limbs 46
labeled-data/spec2_cam0_d/img0750.png 367.38035640584235
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam0_d/img0750.png
annotated limbs 46
labeled-data/spec2_cam0_d/img0780.png 805.0686878468376
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam0_d/img0780.png
annotated limbs 46
labeled-data/spec2_cam0_d/img0782.png 852.903569078633
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam0_d/img0782.png
annotated limbs 46
labeled-data/spec2_cam0_d/img0783.png 871.1038472449952
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam0_d/img0783.png
annotated limbs 46
labeled-data/spec2_cam0_d/img0812.png 1328.9830299786677
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam0_d/

labeled-data/spec2_cam2_h/img0197.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam2_h/img0197.png
annotated limbs 46
labeled-data/spec2_cam2_h/img0242.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam2_h/img0242.png
annotated limbs 46
labeled-data/spec2_cam2_h/img0336.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam2_h/img0336.png
annotated limbs 46
labeled-data/spec2_cam2_h/img0733.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam2_h/img0733.png
annotated limbs 46
labeled-data/spec2_cam2_h/img1013.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam2_h/img1013.png
annotated limbs 46
labeled-data/spec2_cam2_h/img1019.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam2_h/img1019.png
annotated limbs 46
labeled-data/spec2_cam2_h/img1022.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam2_h/img1022.png
annotated limbs 46
labeled-data/spec2_cam2_h/img1127.png 65.4970151779129
I:\BENCHMARK\S

labeled-data/spec2_cam4_h/img1223.png 2025.5600347364732
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam4_h/img1223.png
annotated limbs 46
labeled-data/spec2_cam4_h/img1299.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam4_h/img1299.png
annotated limbs 46
labeled-data/spec2_cam4_h/img1324.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam4_h/img1324.png
annotated limbs 46
labeled-data/spec2_cam4_h/img1343.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam4_h/img1343.png
annotated limbs 46
labeled-data/spec2_cam4_h/img1356.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam4_h/img1356.png
annotated limbs 46
labeled-data/spec2_cam4_h/img1389.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam4_h/img1389.png
annotated limbs 46
labeled-data/spec2_cam4_u/img0118.png 159.9584963124621
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec2_cam4_u/img0118.png
annotated limbs 46
labeled-data/spec2_cam4_u/img0187.png 23

labeled-data/spec3_cam1_d/img237.png 405.80445293014157
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam1_d/img237.png
annotated limbs 46
labeled-data/spec3_cam1_d/img246.png 236.89205172069254
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam1_d/img246.png
annotated limbs 46
labeled-data/spec3_cam1_d/img252.png 23.602322573427315
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam1_d/img252.png
annotated limbs 46
labeled-data/spec3_cam1_d/img257.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam1_d/img257.png
annotated limbs 46
labeled-data/spec3_cam1_d/img265.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam1_d/img265.png
annotated limbs 46
labeled-data/spec3_cam1_d/img275.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam1_d/img275.png
annotated limbs 46
labeled-data/spec3_cam1_i/img055.png 1695.881904693571
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam1_i/img055.png
annotated limbs 46
labeled-data/spec3_cam1_

labeled-data/spec3_cam2_u/img156.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam2_u/img156.png
annotated limbs 46
labeled-data/spec3_cam2_u/img161.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam2_u/img161.png
annotated limbs 46
labeled-data/spec3_cam2_u/img164.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam2_u/img164.png
annotated limbs 46
labeled-data/spec3_cam2_u/img171.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam2_u/img171.png
annotated limbs 46
labeled-data/spec3_cam2_u/img177.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam2_u/img177.png
annotated limbs 46
labeled-data/spec3_cam3_d/img184.png 1752.0397782465116
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam3_d/img184.png
annotated limbs 46
labeled-data/spec3_cam3_d/img191.png 1459.576602312137
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec3_cam3_d/img191.png
annotated limbs 46
labeled-data/spec3_cam3_d/img195.png 1304.116423813673

...

labeled-data/spec6_cam2_i/img0865.png 1262.8792325673005
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam2_i/img0865.png
annotated limbs 46
labeled-data/spec6_cam2_i/img0892.png 1359.0909166885112
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam2_i/img0892.png
annotated limbs 46
labeled-data/spec6_cam2_i/img0913.png 1379.509524213525
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam2_i/img0913.png
annotated limbs 46
labeled-data/spec6_cam2_i/img0925.png 1430.6986421367505
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam2_i/img0925.png
annotated limbs 46
labeled-data/spec6_cam2_i/img0980.png 1630.8457180464081
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam2_i/img0980.png
annotated limbs 46
labeled-data/spec6_cam2_i/img0994.png 1721.299729876716
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam2_i/img0994.png
annotated limbs 46
labeled-data/spec6_cam2_i/img1090.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam2_i/img1090.png


labeled-data/spec6_cam4_d/img0929.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam4_d/img0929.png
annotated limbs 46
labeled-data/spec6_cam4_d/img0945.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam4_d/img0945.png
annotated limbs 46
labeled-data/spec6_cam4_d/img0987.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam4_d/img0987.png
annotated limbs 46
labeled-data/spec6_cam4_i/img0104.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam4_i/img0104.png
annotated limbs 46
labeled-data/spec6_cam4_i/img0114.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam4_i/img0114.png
annotated limbs 46
labeled-data/spec6_cam4_i/img0486.png 73.45451360972618
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam4_i/img0486.png
annotated limbs 46
labeled-data/spec6_cam4_i/img0560.png 155.47665996856514
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec6_cam4_i/img0560.png
annotated limbs 46
labeled-data/spec6_cam4_i/img0629.png 16

I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam0_u/img139.png
annotated limbs 46
labeled-data/spec7_cam0_u/img147.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam0_u/img147.png
annotated limbs 46
labeled-data/spec7_cam0_u/img157.png nan
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam0_u/img157.png
annotated limbs 46
labeled-data/spec7_cam1_i/img001.png 1867.2028998738397
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam1_i/img001.png
annotated limbs 46
labeled-data/spec7_cam1_i/img039.png 1705.4062567161252
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam1_i/img039.png
annotated limbs 46
labeled-data/spec7_cam1_i/img145.png 1817.9227893183972
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam1_i/img145.png
annotated limbs 46
labeled-data/spec7_cam1_i/img283.png 1677.423584000428
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam1_i/img283.png
annotated limbs 46
labeled-data/spec7_cam1_i/img313.png 1451.7730032328427
I:\BENCHM

labeled-data/spec7_cam2_u/img061.png 889.0801679558432
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam2_u/img061.png
annotated limbs 46
labeled-data/spec7_cam2_u/img089.png 1313.3142017333314
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam2_u/img089.png
annotated limbs 46
labeled-data/spec7_cam2_u/img091.png 1353.1410687675411
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam2_u/img091.png
annotated limbs 46
labeled-data/spec7_cam2_u/img096.png 1458.1511417052207
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam2_u/img096.png
annotated limbs 46
labeled-data/spec7_cam2_u/img098.png 1487.4526203719877
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam2_u/img098.png
annotated limbs 46
labeled-data/spec7_cam2_u/img109.png 1691.2337793443799
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam2_u/img109.png
annotated limbs 46
labeled-data/spec7_cam2_u/img113.png 1803.535531739393
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam2_u/img113.png

labeled-data/spec7_cam4_h/img266.png 1113.28000213149
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam4_h/img266.png
annotated limbs 46
labeled-data/spec7_cam4_h/img276.png 1213.73942997584
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam4_h/img276.png
annotated limbs 46
labeled-data/spec7_cam4_h/img281.png 1347.12054901617
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam4_h/img281.png
annotated limbs 46
labeled-data/spec7_cam4_h/img288.png 1460.2070038607599
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam4_h/img288.png
annotated limbs 46
labeled-data/spec7_cam4_h/img294.png 1558.07063741499
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam4_h/img294.png
annotated limbs 46
labeled-data/spec7_cam4_h/img309.png 1833.2563396452
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam4_h/img309.png
annotated limbs 46
labeled-data/spec7_cam4_h/img313.png 1918.9859133543698
I:\BENCHMARK\Sungaya_hand_labeled\labeled-data/spec7_cam4_h/img313.png
annotate

In [21]:
thresh = 0.1
point_size = 5
desired_dims = (1024,768)

dlc_test_image_path = "I:/BENCHMARK/Sungaya_hand_labeled/labeled-data/spec2_cam0_d/img0812.png"

dlc_test_image = cv2.imread(dlc_test_image_path)
dlc_test_image_resized = cv2.resize(dlc_test_image,desired_dims)

dlc_live.init_inference(dlc_test_image_resized)
pose = dlc_live.get_pose(dlc_test_image_resized)

for p, point in enumerate(pose):
    if point[2] >= thresh:
        dlc_test_image_resized = cv2.circle(dlc_test_image_resized, (int(point[0]), int(point[1])),
                                   point_size,
                                   (int(255 * p / 49), int(255 - 255 * p / 49), 200),
                                   -1)
            
cv2.imshow("DLC Test IMG",dlc_test_image_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()